In [24]:
import pandas as pd
from pathlib import Path
import statsmodels.formula.api as sm
import numpy as np

In [25]:
interim = '../data/interim'
br = pd.read_pickle(Path(interim) / 'batting_records.pkl')
gl = pd.read_pickle(Path(interim) / 'game_logs.pkl')
events = pd.read_pickle(Path(interim) / 'events.pkl')
people = pd.read_pickle(Path(interim) / 'people.pkl')

In [16]:
## League DAta
League = events[events.BAT_FLD_CD != 1].groupby('year')[['HR', 'H', 'PA']].sum()
League.columns = ['League_HR', 'League_H', 'League_PA']

In [17]:
## combine data
br = br.reset_index()
merged = pd.merge(br, people[['PlayerID', 'birthYear']], left_on = ['BAT_ID'], right_on=['PlayerID'], how='left')
merged = merged.set_index(['BAT_ID', 'year']).sort_values(['BAT_ID', 'year'], ascending=[True, True])
merged['Age'] = merged.index.get_level_values('year') - merged['birthYear']

merged['PA'] = merged['PA'].astype('int')
merged['H'] = merged['H'].astype('int')
merged['HPPA'] = merged['HPPA'].astype('float')

In [18]:
test = merged
test = test.sort_values(['BAT_ID', 'year'])

metric = 'H'

group = test.groupby('BAT_ID')

test['L1_HPPA'] = group['HPPA'].shift(1)
test['L2_HPPA'] = group['HPPA'].shift(2)
test['L3_HPPA'] = group['HPPA'].shift(3)

test['L1_' + metric] = group[metric].shift(1, fill_value=0)
test['L2_' + metric] = group[metric].shift(2, fill_value=0)
test['L3_' + metric] = group[metric].shift(3, fill_value=0)

test['L1_PA'] = group['PA'].shift(1, fill_value=0)
test['L2_PA'] = group['PA'].shift(2, fill_value=0)
test['L3_PA'] = group['PA'].shift(3, fill_value=0)

In [19]:
# limited = merged[(merged.G >= 50)  & (merged.prev_G >= 50)]
result = sm.ols(formula="PA ~ L1_PA + L2_PA", data=test).fit()
# print(result.summary())
coef_const = result.params[0]
coef_L1_PA = result.params[1]
coef_L2_PA = result.params[2]

result2 = sm.ols(formula="H ~ L1_H + L2_H", data=test).fit()
# print(result2.summary())
coef_const2 = result2.params[0]
coef_L1_PA2 = result2.params[1]

result3 = sm.ols(formula="HPPA ~ L1_HPPA + L2_HPPA + L3_HPPA -1", data=test).fit()
# print(result3.summary())
coef_L1_PA3 = result3.params[0]
coef_L2_PA3 = result3.params[1]
coef_L3_PA3 = result3.params[2]

sum_coef = coef_L1_PA3+ coef_L2_PA3 + coef_L3_PA3
adj = 12 / sum_coef
print(adj)

coef_L1_PA3 = adj * coef_L1_PA3
coef_L2_PA3 = adj * coef_L2_PA3
coef_L3_PA3 = adj * coef_L3_PA3

sum_coef2 = coef_L1_PA3+ coef_L2_PA3 + coef_L3_PA3
print(coef_L1_PA3)
print(coef_L2_PA3)
print(coef_L3_PA3)
print(sum_coef2)

12.84138037125633
4.627599395051298
4.15182994418264
3.2205706607660622
12.000000000000002


In [ ]:
League['League_HPPA'] = League['League_H'] / League['League_PA']
League = League.sort_values('year')
League['exp_League_HPPA'] = (
    coef_L1_PA3 * League['League_HPPA'].shift(1) + 
    coef_L2_PA3 * League['League_HPPA'].shift(2) +
    coef_L3_PA3 * League['League_HPPA'].shift(3)
)/12
League['exp_League_HPPA'] = League['exp_League_HPPA'].astype('float')

In [21]:
test = pd.merge(test, League, on=['year'], how='left')

In [22]:
test

,FirstName,LastName,BAT_HAND,G,AB,H,2B,3B,HR,RBI,...,L2_H,L3_H,L1_PA,L2_PA,L3_PA,League_HR,League_H,League_PA,League_HPPA,exp_League_HPPA
year,,,,,,,,,,,,,,,,,,,,,
2006,David,Aardsma,1.0,3,2,0,0,0,0,0,...,0,0,0,0,0,5354,44403,182052,0.243903,0.239560
2008,David,Aardsma,1.0,1,1,0,0,0,0,0,...,0,0,3,0,0,4860,43275,181711,0.238153,0.242119
2015,David,Aardsma,1.0,1,1,0,0,0,0,0,...,0,0,1,3,0,4883,41482,178223,0.232753,0.230993
1954,Hank,Aaron,1.0,117,449,127,26,6,13,67,...,0,0,0,0,0,1854,20102,84793,0.237071,0.236944
1955,Hank,Aaron,1.0,141,559,172,32,9,25,95,...,0,0,489,0,0,2099,19774,84723,0.233396,0.237368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,Paul,Zuvella,1.0,66,190,48,8,1,0,4,...,0,0,27,8,1,3582,36182,155350,0.232906,0.236587
1986,Paul,Zuvella,1.0,20,48,4,1,0,0,2,...,5,0,210,27,8,3789,36292,155924,0.232754,0.235041
1987,Paul,Zuvella,1.0,13,34,6,0,0,0,0,...,48,5,57,210,27,4438,37300,156983,0.237605,0.233725


In [23]:
test['League_' + metric + 'PPA'] = test['League_' + metric] / test['League_PA']
test['L1_League_' + metric + 'PPA'] = group['League_' + metric + 'PPA'].shift(1, fill_value=0)
test['L2_League_' + metric + 'PPA'] = group['League_' + metric + 'PPA'].shift(2, fill_value=0)
test['L3_League_' + metric + 'PPA'] = group['League_' + metric + 'PPA'].shift(3, fill_value=0)

test['exp_' + metric] = test['PA'] * test['League_' + metric + 'PPA']
test['L1_exp_' + metric] = group['exp_' + metric].shift(1, fill_value=0)
test['L2_exp_' + metric] = group['exp_' + metric].shift(2, fill_value=0)
test['L3_exp_' + metric] = group['exp_' + metric].shift(3, fill_value=0)

KeyError: 'Column not found: League_HPPA'

In [ ]:
merged = pd.merge(merged, League, on=['year'], how='left')

In [30]:
test['exp_PA'] = coef_L1_PA * test['L1_PA'] + coef_L2_PA * test['L2_PA'] + coef_const

test['sum_' + metric] = (
    coef_L1_PA3 * test['L1_' + metric] + 
    coef_L2_PA3 * test['L2_' + metric] + 
    coef_L3_PA3 * test['L3_' + metric]
)
test['sum_PA'] = (
    coef_L1_PA3 * test['L1_PA'] + 
    coef_L2_PA3 * test['L2_PA'] + 
    coef_L3_PA3 * test['L3_PA']
)
test['exp_sum_' + metric] = (
    coef_L1_PA3 * test['L1_exp_' + metric] +
    coef_L2_PA3 * test['L2_exp_' + metric] +
    coef_L3_PA3 * test['L3_exp_' + metric]
)

test['exp_' + metric + '_per_1200'] = (test['exp_sum_' + metric] / test['sum_PA']) * 1200
test['exp_' + metric + '_per_pa'] = np.where(
    test['sum_PA'] > 0,
    (test['exp_' + metric + '_per_1200'] + test['sum_' + metric]) / (1200 +  test['sum_PA']),
    test['exp_League_' + metric + 'PPA']
).astype('float')

test['age_adj'] = np.where(
    test['Age'] <= 29, 
    1 + (29 - test['Age'])* .006,
    1 + (29 - test['Age'])* .003,
)

test['adj_exp_' + metric + '_per_pa'] = test['exp_' + metric + '_per_pa'] * test['age_adj']
test['adj_exp_' + metric] = test['adj_exp_' + metric + '_per_pa'] *  test['exp_PA']

merged['pred_HPPA'] = test['adj_exp_H_per_pa'].astype('float')

merged = merged.sort_values(['BAT_ID', 'year'])
merged['prev_G'] = merged.groupby('BAT_ID')['G'].shift(1)

merged = merged[['BAT_HAND', 'pred_HPPA', 'G', 'prev_G', 'H', 'PA']]
merged['HPPA'] = (merged['H'] / merged['PA']).astype('float')

In [31]:
limited = merged[(merged.G >= 50)  & (merged.prev_G >= 50)]
result = sm.ols(formula="HPPA ~ pred_HPPA", data=limited).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   HPPA   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     9271.
Date:                Sun, 21 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:51:13   Log-Likelihood:                 47369.
No. Observations:               21755   AIC:                        -9.473e+04
Df Residuals:                   21753   BIC:                        -9.472e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0276      0.002     12.679      0.0